# HQ Protocol

## Description

The HighQ Protocol is used to communicate with the laser or with one of its
components. A communication is initiated by a dedicated master and is then
answered by the addressed slave.

## Communication parameters
* 4800 Baud
* 8 Bit
* No Parity
* 1 Stop Bit

Commonly used is the RS422 interface. Other interfaces are also possible.

## Packet definition


| Index | Size | Name | Description
|---|---|---|---|
| &nbsp; | 1 | SYN | Synchronization Byte (0x16) |
| 0 | 1 | STX | Start Byte (0x02) |
| 1 | 1 | LEN | Message Data Length N + 7 (for other fields except SYN) |
| 2 | 1 | SRC | ID of the Source Device |
| 3 | 1 | DST | ID of the Destination Device |
| 4 | 1 | CMD | Command Byte |
| 5 | N | DATA | Data Bytes, Maximum is 32 bytes |
| 5+N | 2 | CRC | Checksum IBM CRC16 (high byte first) |

### SYN
The synchronization byte is send before the actual packet and is not
part of the packet itself. So it doesn't count for the packet length
(LEN) and the CRC. Sync bytes are usually defined as optional and if they
are used there could also be multiple ones, BUT in the HighQ protocol has
to be exactly one sync byte before the actual packet.

### STX
The start byte is the first byte in the packet and signalizes the
start of the packet.

### LEN
The length byte specifies the length of the packet. Counted are
the following bytes: STX, LEN, SRC, DST, CMD, N DATA bytes and
2 bytes CRC. So this length is the amount of data bytes plus 7.
There is a restriction to maximum 32 data bytes, so the maximum
value for the LEN byte is 39.

### SRC
Source ID. Master has the source id 0, if it is a request message. The
answer message has the source id of the addressed slave.

### DST
Destination ID. Addressing the slave in a request message. The answer
message has usually a destination ID 0 (addressing the master).
If the request has a destination ID of 255, every slave is addressed.
This could be used, if only one slave is connected to a master and the
ID is not known.

### CMD
Specifies the requested command the slave should process. The answer
message from the slave uses the same value. Some command values are
predefined and listed below.

### DATA
Contains the message data for this particular command. In the request
message the master sends data to the slave and vice versa for the answer.
Some requests or answers will have no data. The data is usually represented
as big endian (high bytes first).

### CRC
The CRC is calculated from STX to the end of DATA (or to CMD if no DATA is
available). The used CRC polynom is IBM-CRC-16 (x^16 + x^15 + x^2 + 1),
start value is 0.

## Request Sequence
A request is initated by the PC using the source id `0`. Use the command byte to send
the requested command. May add additional data in the data field. The destionation field
contains the id of the selected slave device.

Example of a request:

    PC -> Slave: 16 02 07 00 02 50 E8 79
    Slave -> PC: 16 02 07 02 00 50 48 D9

In this example the PC send a request to slave id 2 requesting the command 0x50. The slave answers with a packet containing the same command byte and no additional data.

## Usage of hq_protocol c module

### include hq_protocol.h

    #include "hq_protocol.h"

### instantiate a `hq_protocol_stack_t` instance

    hq_protocol_stack_t stack;

### initialize the stack and define two functions

`uint8_t send_byte(hq_protocol_stack_t *stack, uint8_t byte)`

This function should send the given byte over the serial interface. `stack` is
used for differentiation if more than one connection is handled.

Return value should be 0, is sending was OK. If a problem occured return 1

`void process_packet(hq_protocol_stack_t *stack, hq_protcol_packet_t *packet)`

When a packet was received by the stack this function will be called. See
the definition of hq_protocol_packet_t for its members.

### initialize stack and set function pointers

    void init(void) {
      hq_protocol_init(&stack);
      stack.send_byte = send_byte;
      stack.process_packet = process_packet;
    }

### send a request by using `hq_protocol_send_packet_assembled`

e.g. sending a string "Hello" to ID 1 via cmd is 0x20 (source ID will be 0):

    hq_protocol_send_packet_assembled(&stack, 0, 1, 0x20, 5, "Hello");

Running this function `send_byte(...)` will be called and will send the bytes
via serial interface.

### call `hq_protocol_process_rx_byte` on each received byte

See definition of hq_protocol_process_rx_byte for the return values. When a
packet was received the function `process_packet` will be called.

## Interactive Python examples
### Calculating CRC

In [1]:
def calc_crc(data: bytes, crc: int=0) -> int:
    """Calulates 16 bit crc IBM (polynom x^16+x^15+x^2+1)

    :param data: byte string
    :param crc: initial value of crc
    """ 
    for byte in data: # for each byte in array
        for i in range(8):   # for each bit in byte
            if ((byte^crc)&0x01):
                crc=((crc>>1) ^ 0xA001)
            else:
                crc=(crc >> 1)
            byte=byte>>1
    return crc

Calculating the crc of a given packet. Be aware that the SYNC byte is **not** included in this calculation!

In [2]:
base_packet_bstr = b'\x02\x07\x00\x02\x50'
calc_crc(base_packet_bstr)

59513

So the complete request to send is in this case:

In [3]:
import struct

def bytes_to_str(bstr: bytes) -> str:
    """ Helper function to pretty print a bytes array as string with hex values."""
    return " ".join(["{:02x}".format(b) for b in bstr])

bytes_to_str(b'\x16' + base_packet_bstr + struct.pack('>H', calc_crc(base_packet_bstr)))

'16 02 07 00 02 50 e8 79'

### Creating a request packet

In [4]:
def build_packet(destination_id: int, command: int, data: bytes, source_id: int = 0):
    """ Bulding a HighQ protocol packet """
    packet = b'\x02' + struct.pack('>BBBB', len(data)+7, source_id, destination_id, command) + data
    return b'\x16' + packet + struct.pack('>H', calc_crc(packet))

Example usage with following packet:

* Destination ID is 7
* Command is 0x20
* Data to be send is a 16 bit integer with value 1000

In [5]:
packet_bstr = build_packet(destination_id=7, command=0x20, data=struct.pack('>H', 1000))
bytes_to_str(packet_bstr)

'16 02 09 00 07 20 03 e8 59 23'

### Interactive creation of packet

In [6]:
from ipywidgets import interact

@interact(source_id=(0, 255), destination_id=(0, 255), command=(0, 255), data_str='00 00')
def build_interactive_packet(source_id=0, destination_id=7, command=0x20, data_str=''):
    try:
        data = bytes.fromhex(data_str)
    except ValueError:  # this can be raised during editing
        data =b''
    packet = build_packet(destination_id, command, data, source_id=source_id)
    return bytes_to_str(packet)

interactive(children=(IntSlider(value=0, description='source_id', max=255), IntSlider(value=7, description='de…

### Parsing a packet

In [7]:
from collections import namedtuple

Packet = namedtuple('Packet', 'source_id destination_id command data')

def parse_packet(received_bytes: bytes) -> Packet:
    """ This is a rather naive approach for parsing a packet but it shows how it works """
    start_index = received_bytes.find(b'\x16\x02')
    if start_index < 0:
        return 'No SYNC and START byte found'
    
    if len(received_bytes) < start_index + 8:
        return 'Not enough bytes received'
    
    length = received_bytes[start_index + 2]
    
    if length < 7 or len(received_bytes) < start_index + length + 1:
        return 'Length byte invalid or too few data bytes received'
    
    source_id = received_bytes[start_index + 3]
    destination_id = received_bytes[start_index + 4]
    command = received_bytes[start_index + 5]
    data = received_bytes[start_index + 6: start_index + length - 1]
    crc = struct.unpack('>H', received_bytes[start_index + length - 1: start_index + length + 1])[0]
    
    if crc != calc_crc(received_bytes[start_index + 1: start_index + length - 1]):
        return 'Wrong CRC'
    
    return Packet(source_id, destination_id, command, data)
    

parse_packet(b'\x16\x02\x09\x07\x00\x20\x00\x00\x53\x97')
    

Packet(source_id=7, destination_id=0, command=32, data=b'\x00\x00')

### Interactive parsing of bytes

In [8]:
@interact(received_str='16 02 09 07 00 20 00 00 53 97')
def interactive_parse_packet(received_str):
    try:
        received_bytes = bytes.fromhex(received_str)
    except ValueError:  # this can be raised during editing
        received_bytes =b''
        
    result = parse_packet(received_bytes)
    
    if isinstance(result, Packet):
        print('Packet receceived with source id %d, command id 0x%02X and data %r' % (result.source_id, result.command, result.data))
    else:
        print(result)

interactive(children=(Text(value='16 02 09 07 00 20 00 00 53 97', description='received_str'), Output()), _dom…